## TensorFlow TensorBoard计算图可视化

选择环境：Anaconda Python 3.6.6  
安装Tensorflow：Python 3.6环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

## 11.1 TensorBoard简介

TensorBoard可以有效地展示TensorFlow在运行过程中的计算图、各种指标随着时间的变化趋势以及训练中使用到的图像等信息。TensorBoard和TensorFlow程序跑在不同的进程中，TensorBoard会自动读取最新的TensorFlow日志文件，并呈现当前TensorFlow程序运行的最新状态。

In [1]:
import os
os.chdir('C:/Users/74575/Desktop/pythondata')
print(os.getcwd())

C:\Users\74575\Desktop\pythondata


In [2]:
import tensorflow as tf

# 定义一个简单的计算图，实现向量加法的操作
input1 = tf.constant([1.0, 2.0, 3.0], name='input1')
input2 = tf.Variable(tf.random_uniform([3]), name='input2')
output = tf.add_n([input1, input2], name='add')

# 生成一个写日志的Writer，并将当前的TensorFlow计算图写入日志
# TensorFlow提供了很多种写日志文件的API，11.3节会详细介绍
writer = tf.summary.FileWriter("log", tf.get_default_graph())
writer.close()

以上程序输出了TensorFlow计算图的信息。  
TensorFlow安装完成时，TensorBoard会自动安装。运行以下命令便可以启动TensorBoard:

`
tensorboard --logdir=/path/to/log
`

运行以上命令会启动一个服务，这个服务的端口默认为**6006**。通过浏览器打开localhost:6006，在界面的上方，展示的内容是“GRAPHS”，表示图中可视化的内容是TensorFlow的计算图。打开TensorBoard界面会默认进入GRAPHS界面，在该界面中可以看到上面程序TensorFlow计算图的可视化结果。

上方有一个“INACTIVE＂选项，点开这个选项可以看到TensorBoard能够可视化的其他内容。“INACTIVE”选项中列出的是当前没有可视化数据的项目。除了可视化TensorFlow计算图之外， TensorBoard还提供了SCALARS、IMAGES、AUDIO、DISTRIBUTIONS、HISTOGRAMS、PROJECTOR、TEXT和PROFILE项目。TensorBoard中每一栏都对应了一类信息的可视化结果。

In [ ]:
# 在log的母文件夹，打开cmd，输入activate py36，运行该命令
tensorboard --logdir=log

## 11.2 TensorBoard计算图可视化

当神经网络模型的结构更加复杂、运算更多时，其所对应的TensorFlow计算图会复杂很多。为了更好地组织可视化效果图中的计算节点，TensorBoard支持通过TensorFlow命名空间来整理可视化效果图上的节点。在TensorBoard的默认视图中，TensorFlow计算图中同一个命名空间下的所有节点会被缩略成一个节点，只有顶层命名空间中的节点才会被显示在TensorBoard可视化效果图上。

前面已经介绍过变量的命名空间，以及如何通过`tf.variable_scope`函数管理变量的命名空间。除此之外，`tf.name_scope`函数也提供了命名空间管理的功能。这两个函数在大部分情况下是等价的，唯一的区别是在使用`tf.get_variable`函数时。以下代码简单地说明了这两个函数的区别:

In [3]:
import tensorflow as tf

# 1. 不同的命名空间
with tf.variable_scope("foo"):
    # 在命名空间foo下获取变量bar，于是得到的变量名为:foo/bar
    a = tf.get_variable("bar", [1])
    print(a.name)                                   # 输出：foo/bar:0

with tf.variable_scope("bar"):
    # 在命名空间bar下获取变量bar，于是得到的变量名为:bar/bar
    # 此时变量bar/bar和变量foo/bar并不冲突，于是可以正常运行
    b = tf.get_variable("bar", [1])
    print(b.name)                                  # 输出：bar/bar:0
    
# 2. tf.Variable和tf.get_variable的区别
with tf.name_scope("a"):
    # 使用tf.Variable函数生成变量会受tf.name_scope影响
    a = tf.Variable([1])
    print(a.name)                                 # 输出：a/Variable:0
    
    # 使用tf.get_variable函数不受tf.name_scope影响
    a = tf.get_variable("b", [1])
    print(a.name)                                 # 输出：b:0
    
# 由于tf.get_varibale不受name_scope影响，所以这里会报声明重复错误
# with tf.name_scope("b"):
#     tf.get_variable("b", [1])

foo/bar:0
bar/bar:0
a/Variable:0
b:0


通过以下代码，可以改进向量相加的样例代码，使得可视化得到的效果图更加清晰：

In [6]:
import tensorflow as tf

# 将输入定义放进各自的命名空间，从而使得TensorBoard可以根据
# 命名空间来整理可视化效果图上的节点
with tf.name_scope("input1"):
    input1 = tf.constant([1.0, 2.0, 3.0], name="input2")
with tf.name_scope("input2"):
    input2 = tf.Variable(tf.random_uniform([3]), name="input2")
output = tf.add_n([input1, input2], name="add")

writer = tf.summary.FileWriter("log", tf.get_default_graph())
writer.close()

可以看出用于初始化的节点已经被缩略起来了，需要查看input2节点中具体包含了哪些运算时，可以将鼠标移动到input2节点，并点开右上角的加号。

下面将给出一个样例程序来展示如何很好地可视化一个真实的神经网络结构图。继续采用5.5节中给出的架构，以下代码给出了改造后的mnist_train.py程序:

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import mnist_inference

# 1. 定义神经网络的参数
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 3000
MOVING_AVERAGE_DECAY = 0.99


# 2. 定义训练的过程并保存TensorBoard的log文件
def train(mnist):
    #  输入数据的命名空间。
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name='y-input')
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = mnist_inference.inference(x, regularizer)
    global_step = tf.Variable(0, trainable=False)
    
    # 处理滑动平均的命名空间。
    with tf.name_scope("moving_average"):
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
        variables_averages_op = variable_averages.apply(tf.trainable_variables())
   
    # 计算损失函数的命名空间。
    with tf.name_scope("loss_function"):
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
        cross_entropy_mean = tf.reduce_mean(cross_entropy)
        loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    
    # 定义学习率、优化方法及每一轮执行训练的操作的命名空间。
    with tf.name_scope("train_step"):
        learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            mnist.train.num_examples / BATCH_SIZE,
            LEARNING_RATE_DECAY,
            staircase=True)

        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

        with tf.control_dependencies([train_step, variables_averages_op]):
            train_op = tf.no_op(name='train')
    
    # 训练模型。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})

            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
    
    # 将当前计算图中输出到TensorBoard日志文件
    writer = tf.summary.FileWriter("log/mnist.log", tf.get_default_graph())
    writer.close()
    
    
# 3. 主函数
def main(argv=None): 
    mnist = input_data.read_data_sets("../../datasets/MNIST_data", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    tf.app.run()

*上面一个cell中程序在jupyter中会一直报错，可选择使用.py来运行（文件见同目录mnist.py）。*

以上程序最大的改变就是将完成类似功能的计算放到了由`tf.name_scope`函数生成的上下文管理器中。这样TensorBoard可以将这些节点有效地合并，从而突出神经网络的整体结构。

节点之间有两种不同的边：一种边是通过实线表示的，这种边刻画了数据传输，边上箭头方向表达了数据传输的方向。效果图上边的粗细表示的是两个节点之间传输的标量维度的总大小，而不是传输的标量个数。另外一种边是通过虚线表示的，表达了计算之间的依赖关系。

除了手动的通过TensorFlow中的命名空间来调整TensorBoard的可视化效果图，TensorBoard也会智能地调整可视化效果图上的节点。TensorBoard将TensorFlow计算图分成了主图（Main Graph）和辅助图（Auxiliary nodes）。除了自动的方式，TensorBoard也支持手工的方式来调整可视化结果。右键单击可视化效果图上的节点会弹出一个选项，这个选项可以将节点加入主图或者从主图中删除。左键选择一个节点并点击信息框下部的选项也可以完成类似的功能。**注意TensorBoard不会保存用户对计算图可视化结果的手工修改，页面刷新之后计算图可视化结果又会回到最初的样子。**

除了展示TensorFlow计算图的结构，TensorBoard还可以**展示TensorFlow计算图上每个节点的基本信息以及运行时消耗的时间和空间。**可以非常直观地展现所有TensorFlow计算节点在某一次运行时所消耗的时间和内存。将以下代码加入修改后的mnist_train.py神经网络训练部分，就可以将不同迭代轮数的每个TensorFlow计算节点的运行时间和消耗的内存写入TensorBoard的日志文件中。

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in range(TRAINING_STEPS):
        xs, ys = mnist.train.next_batch(BATCH_SIZE)
        if i % 1000 == 0:
            # 配置运行时需要记录的信息
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            # 运行时记录运行信息的proto
            run_metadata = tf.RunMetadata()
            # 将配置信息和记录运行的proto传入运行的过程，从而记录运行时每一个节点的时间、内存信息
            _, loss_value, step = sess.run([train_op, loss, global_step],
                                            feed_dict={x: xs, y_:ys},
                                            options=run_options,
                                            run_metadata=run_metadata)
            # 将节点在运行时的信息写入日志文件
            train_writer.add_run_metadata(run_metadata, 'step%03d'%i)
            print("After %d training step(s), loss on training batch is %g" % (step, loss_value))
    else:
        _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_:ys})

再次运行文件（见同目录mnist_info.py），并使用这个程序输出的日志启动TensorBoard，这样就可以可视化每个TensorFlow计算节点在某一次运行时所消耗的时间和空间。

在Color栏中：
- **Compute time**。可以看到在这次运行中每个TensorFlow计算节点的运行时间；
- **Memory**。可以看到这次运行中每个TensorFlow计算节点所消耗的内存；
- **Structure**。前面图中展示的可视化效果图都是使用默认的Structure选项。在这个视图中，灰色的节点表示没有其他节点和它拥有相同结构。如果有两个节点的结构相同，那么它们会被涂上相同的颜色；
- **Device**。这个选项可以根据TensorFlow计算节点运行的机器给可视化效果图上的节点染色。在使用GPU时，可以通过这种方式直观地看到哪些计算节点被放到了GPU上。

信息卡片：当点击TensorBoard可视化效果图中的节点时，界面的右上角会弹出一个信息卡片显示这个节点的基本信息。当点击节点为一个命名空间时，TensorBoard展示的信息卡片有这个命名空间内所有计算节点的输入、输出以及依赖关系。虽然属性（attributes)也会展示在卡片中，但是在代表命名空间的属性下不会有任何内容。当Session runs选择了某一次运行时，节点的信息卡片上也会出现这个节点运行时所消耗的时间和内存等信息。